# Readme: how to handle our Sentinel-1 ASC-DESC dataset

###What you have: 
A list of big geotiff file (.tif) Pairs. Each image in the pair should roughly correspond to the same area as the other. Each geotiff can by read using rasterio or gdal, as you can see in the notebook.

###What you should do: 
create single datapoints to train/test your models.

###How to do it: 
For each big pair, get a datapoint by cropping a random patch from one side (this acts as the "query" patch, the one you want to look for) and cropping a patch around the same coordinates from the other side (this acts as the "search" patch, the one to want to search in). Add distortions: search patch has to be taken with a random offset around query patch (perhaps random but normally distributed around offset 0, which means query patch at the centre). Also add a bit of random scale and rotation.Try finding a 1km^2 query patch inside a 4km^2 search patch. 

###Some WARNINGS (not too sure about them)

Warning: do not include every correspondence in the ground truth pixel correspondence list! you should only insert meaningful correspondences such as harris corners.

Warning: maybe discard a candidate query altogether if it's not feature-rich enough. You can do this using pixel histograms (has to have more than n peaks -> use scikit learn), or maybe entropy. for example, if every pixel is around e.g. 150, this means that everything is gray and unmatchable.

In [1]:

from scipy.ndimage import uniform_filter, variance
from skimage.transform import AffineTransform


import PIL
from PIL import Image, ImageFilter

import rasterio as rio
from rasterio import warp

import matplotlib

from matplotlib import pyplot as plt

import argparse


import pytorch_lightning as pl



import pandas as pd

# Need this to plot in HD
# This takes up a lot of memory!
matplotlib.rcParams['figure.dpi'] = 300

if 'model_TransSAR' not in locals():
    model_TransSAR=None

2023-02-27 12:30:29.392632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 12:30:30.361770: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-27 12:30:30.361928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-27 12:30:30.361934: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def get_correspondence_multi(src_data, dst_data, src_row, src_col):
  ''' Gets the pixel coordinates (dst_row,dst_col) in dst_data of a given pixel at (src_row,src_col) in src_data.
      Warning: no handling of pixel being outside dst_data! You have to handle that yourself.
      TODO: Should add a check to only get the correspondences with high enough cornerness on both sides
      (because we should only insert meaningful correspondences in the list!).
  '''
  # Get geo coords of pixel from src dataset
  X1, Y1 = rio.transform.xy(src_data.transform, src_row, src_col)
  # print("Geographic coordinates in crs 1: ",X1,Y1)

  if src_data.crs != dst_data.crs:
    # convert coordinates in the crs of the dst dataset
    X2, Y2 = warp.transform(src_data.crs, dst_data.crs, X1, Y1)
  else:
    # if the crs is the same, do nothing
    X2, Y2 = X1, Y1

  # Get corresponding px coords in dst dataset
  # It still returns an index even if out of bounds
  dst_row, dst_col = rio.transform.rowcol(dst_data.transform, X2, Y2)
  # print("Corresponding pixel coordinates in image 2: ",dst_row,dst_col)
  return dst_row, dst_col

def get_correspondence(src_data, dst_data ,src_row, src_col):
  ''' Gets the pixel coordinates (dst_row,dst_col) in dst_data of a given pixel at (src_row,src_col) in src_data.
      Warning: no handling of pixel being outside dst_data! You have to handle that yourself.
      TODO: Should add a check to only get the correspondences with high enough cornerness on both sides 
      (because we should only insert meaningful correspondences in the list!).
  '''
  # Get geo coords of pixel from src dataset
  X1, Y1 = src_data.xy(src_row, src_col)
  X1,Y1 = X1 if type(X1) is list else [X1], Y1 if type(Y1) is list else [Y1]
  #print("Geographic coordinates in crs 1: ",X1,Y1)

  if src_data.crs != dst_data.crs:
    # convert coordinates in the crs of the dst dataset
    X2, Y2 = warp.transform(src_data.crs, dst_data.crs,X1,Y1)
    X2 = X2[0]
    Y2 = Y2[0]
  else: 
    # if the crs is the same, do nothing
    X2, Y2 = X1, Y1

  # Get corresponding px coords in dst dataset
  # It still returns an index even if out of bounds
  dst_row, dst_col = dst_data.index(X2, Y2)
  dst_row, dst_col =  dst_row[0] if type(dst_row) is list else dst_row, dst_col[0] if type(dst_col) is list else dst_col
  if dst_row < 0 or dst_col <0:
      print("make the margin higher, the corresponding points are outside the image")
  #print("Corresponding pixel coordinates in image 2: ",dst_row,dst_col)
  return dst_row , dst_col

def get_datapoint(ref_data,query_data):
  ''' TODO: Call this method on a pair of rasterio datasets. It will generate a random datapoint consisting
      of a smaller SAR patch from the query dataset, and a bigger SAR patch from the reference dataset. 
      The search patch contains the area of the smaller patch, with a random offset. 
  '''
  datapoint = None
  return datapoint

def is_good_patch(patch):
  ''' TODO: Checks if the random query patch is sufficiently texture-rich to be used to train/test the matching model.
      If the patch is too plain (e.g. sea or desert), returns False. This function should also be used at test time: 
      if a sensor image is too plain, there's no need to match it. 
      At test time, something similar should also be done on the reference side. 
  '''
  good = None
  return good

def get_keypoints(patch):
  ''' TODO: Use something like harris corner detection to get the list of ground truth correspondences.
      In fact, you should not train on each pixel corresp, but you should select only meaningful corresp!
      Harris peaks might be just strong speckle noise, but if you take strong ones you should be fine.
  '''
  keypoints = None
  return keypoints

def normalize_image(img):
  ''' Normalize by clipping to 99th percentile and convert to uint8.
      This clips strong speckle outliers and optimizes the brightness range
  '''
  p1 = np.nanpercentile(img,99)
  img = img.clip(0,p1)
  img = (img-np.nanmin(img))/(np.nanmax(img)-np.nanmin(img))*255
  img = img.astype(np.uint8, copy=True)
  return img

In [3]:
paths = ['./data/paired_sentinel/Sentinel1-AD Dataset- Lat-0.015502064951149919Lon41.3552557262833/S1A_IW_GRDH_1SDV_20220808T060138_20220808T060203_044457_054E18_B16D.tif',
         './data/paired_sentinel/Sentinel1-AD Dataset- Lat-0.015502064951149919Lon41.3552557262833/S1A_IW_GRDH_1SDV_20220814T175523_20220814T175548_044552_055140_2AEA.tif']


import random
def get_sample(tiff1_path,tiff2_path,search_window,patch_size,margin=40,random_seed=1,verbose=True,random_rotation=0.03,random_zoom=0.03):
  '''
        1-Reads the first band of the TIFF files using the rio library and normalizes the image data.
        2-Selects a random search window and patch location within the image using random number generators.
        3-Calls the get_correspondence function on the selected locations to find the corresponding locations in the second image.
        4-Converts the grayscale images to RGB format using OpenCV.
        5-Creates centered patches from the RGB images by zero-padding the images and copying a portion of the original images to the patches.
        6-Draws rectangles around the search window and patch in both images.
        7-Saves the patch and search window as JPEG files.
        8-Returns the processed RGB images, the points of the search window and patch, and the original rio datasets.

  '''
  dataset1 = rio.open(tiff1_path)
  dataset2 = rio.open(tiff2_path)
  img1 = dataset1.read(1)
  img1 = normalize_image(img1)
  img2 = dataset2.read(1)
  img2 = normalize_image(img2)

  img1= np.swapaxes(img1,0,1)
  img2= np.swapaxes(img2,0,1)

  search_window_w,search_window_h = search_window
  patch_size_w,patch_size_h = patch_size

  if img1.shape[0]-search_window_w-margin*2 <0  or img1.shape[1]-search_window_h-margin*2 <0:
    print("margin + search windows is too big for the image:",margin,"*2 +",(search_window_w,search_window_h),">",img1.shape)
    return None,None,None,None,None,None,None
  lu = ( margin+random.randint(0,img1.shape[0]-search_window_w-margin*2),margin+random.randint(0,img1.shape[1]-search_window_h-margin*2))
  lu_patch=( lu[0] + random.randint(0,search_window_w-patch_size_w),lu[1] + random.randint(0,search_window_h-patch_size_h))


  points_patch = lu_patch
  points = lu
  #print(points,"points",img1.shape)
  points_ref = get_correspondence(dataset1, dataset2,lu[0],lu[1]  )
  points_patch_ref = get_correspondence(dataset1, dataset2,lu_patch[0] ,lu_patch[1] )

  rgb_img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2RGB)
  rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2RGB)

  patch_source = np.zeros((search_window_w, search_window_h), dtype = np.uint8)

  patch_source[int(search_window_w/2 - patch_size_w/2):int(search_window_w/2 + patch_size_w/2),int(search_window_h/2 - patch_size_h/2):int(search_window_h/2 + patch_size_h/2)] = img1[points_patch[0]:points_patch[0]+patch_size_w,points_patch[1]:points_patch[1]+patch_size_h]


  patch_dest = np.zeros((search_window_w, search_window_h), dtype = np.uint8)
  patch_dest[int(search_window_w/2 - patch_size_w/2):int(search_window_w/2 + patch_size_w/2),int(search_window_h/2 - patch_size_h/2):int(search_window_h/2 + patch_size_h/2)]=img2[points_patch_ref[0]:points_patch_ref[0]+patch_size_w,points_patch_ref[1]:points_patch_ref[1]+patch_size_h]



  search_window_source = img1[points[0]:points[0]+search_window_w,points[1]:points[1]+search_window_h]
  search_window_dest = img2[ points_ref[0]:points_ref[0]+search_window_w,points_ref[1] : points_ref[1]+search_window_h]


  Image.fromarray(cv2.cvtColor(patch_dest, cv2.COLOR_GRAY2RGB), "RGB").save("patch.jpeg")
  Image.fromarray(cv2.cvtColor(search_window_source, cv2.COLOR_GRAY2RGB), "RGB").save("search_window.jpeg")

    #draw searching windows




  rgb_img1  = cv2.rectangle(rgb_img1 , tuple(reversed(points)),(points[1]+search_window_h ,points[0]+search_window_w), (0,0,255), 2)
  rgb_img1  = cv2.rectangle(rgb_img1 , tuple(reversed(points_patch)), (points_patch[1]+patch_size_h, points_patch[0]+patch_size_w), (255,0,0), 2)


  #draw patch windows
  rgb_img2  = cv2.rectangle(rgb_img2 , tuple(reversed(points_ref)), ( points_ref[1]+search_window_h,points_ref[0]+search_window_w), (0,0,255), 2)
  rgb_img2  = cv2.rectangle(rgb_img2 , tuple(reversed(points_patch_ref)), ( points_patch_ref[1]+patch_size_h,points_patch_ref[0]+patch_size_w), (255,0,0), 2)

  #print(rgb_img1.shape,search_window_source.shape)


  rgb_img1= np.swapaxes(rgb_img1,0,1)
  rgb_img2= np.swapaxes(rgb_img2,0,1)
  search_window_source= np.swapaxes(search_window_source,0,1)
  patch_source= np.swapaxes(patch_source,0,1)
  search_window_dest= np.swapaxes(search_window_dest,0,1)
  patch_dest= np.swapaxes(patch_dest,0,1)
  if verbose:
      fig, axes = plt.subplots(2, 3)
      axes[0, 0].set_title('source image')
      print(rgb_img1.shape)
      axes[0, 0].imshow(PIL.ImageOps.invert(  Image.fromarray(rgb_img1)))


      axes[0, 1].set_title('search window source')
      axes[0, 1].imshow(PIL.ImageOps.invert(  Image.fromarray( cv2.cvtColor(search_window_source, cv2.COLOR_GRAY2RGB))))

      axes[0, 2].set_title('patch source')
      axes[0, 2].imshow(PIL.ImageOps.invert(  Image.fromarray(cv2.cvtColor(patch_source, cv2.COLOR_GRAY2RGB))))

      axes[1, 0].set_title('dest image')
      axes[1, 0].imshow(PIL.ImageOps.invert(  Image.fromarray(rgb_img2)))

      axes[1, 1].set_title('search window dest')
      axes[1, 1].imshow(PIL.ImageOps.invert(  Image.fromarray(cv2.cvtColor(search_window_dest, cv2.COLOR_GRAY2RGB))))

      axes[1, 2].set_title('patch dest')
      axes[1, 2].imshow(PIL.ImageOps.invert(  Image.fromarray(cv2.cvtColor(patch_dest, cv2.COLOR_GRAY2RGB))))

      plt.show()
  rgb_img1= np.swapaxes(rgb_img1,0,1)
  rgb_img2= np.swapaxes(rgb_img2,0,1)

  return rgb_img1, rgb_img2, points,points_patch_ref,points_patch,dataset1,dataset2


In [4]:
import torch
import cv2
import numpy as np
import matplotlib.cm as cm


def parse_args():
    # init a costum parser which will be added into pl.Trainer parser
    # check documentation: https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags
    parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument(
        'data_cfg_path', type=str,default=".", help='data config path')
    parser.add_argument(
        '--ckpt_path', type=str, default="weights/indoor_large-SEA.ckpt", help='path to the checkpoint')
    parser.add_argument(
        '--dump_dir', type=str, default=None, help="if set, the matching results will be dump to dump_dir")
    parser.add_argument(
        '--profiler_name', type=str, default='inference', help='options: [inference, pytorch], or leave it unset')
    parser.add_argument(
        '--batch_size', type=int, default=1, help='batch_size per gpu')
    parser.add_argument(
        '--num_workers', type=int, default=2)
    parser.add_argument(
        '--thr', type=float, default=None, help='modify the coarse-level matching threshold.')

    parser = pl.Trainer.add_argparse_args(parser)
    return parser.parse_args()

from src.loftr import LoFTR, default_cfg

from src.utils.plotting import make_matching_figures
from src.config.default import get_cfg_defaults
def get_matcher(image_type):

    matcher = LoFTR(config=default_cfg)
    if image_type == 'indoor':
      matcher.load_state_dict(torch.load("weights/indoor_ds.ckpt")['state_dict'])
    elif image_type == 'outdoor':
      matcher.load_state_dict(torch.load("weights/outdoor_ds.ckpt")['state_dict'])
    #model = PL_LoFTR(config, pretrained_ckpt=args.ckpt_path, profiler=profiler)
    return matcher.eval().cuda()

def get_metrics( mkpts0_r, mkpts1_r,points,points_patch,dataset1,dataset2,searching_window_w,searching_window_h,patch_w,patch_h,img0_raw,img1_raw,color,verbose=False):



                # Extract x and y coordinates from mkpts0
                x0, y0 = zip(*mkpts0_r)

                # Apply translation to the coordinates
                x_conv, y_conv = get_correspondence_multi(dataset1, dataset2, [x+points[1] for x in x0], [y+points[0] for y in y0])

                # Convert the translated coordinates back to tuples
                mk0 = list(zip(x_conv, y_conv))

                # Calculate RMSE
                rmse = 0
                for (x1, y1), (x2, y2) in zip(mk0, mkpts1_r):
                    y2 -= (searching_window_w/2)-(patch_w/2)
                    x2 -= (searching_window_h/2)-(patch_h/2)
                    y2 += points_patch[0]
                    x2 += points_patch[1]

                    rmse += ((x1-x2)**2 + (y1-y2)**2)

                # Normalize the errors by the number of keypoints
                num_kpts = len(mk0)
                rmse=  (rmse/num_kpts)**(1/2) # like in "A Transformer-Based Coarse-to-Fine Wide-Swath SAR Image Registration Method under Weak Texture Conditions"

                return rmse
from src.utils.plotting import make_matching_figure
def predict_and_print(rgb_img1,rgb_img2,matcher,img0_raw,img1_raw,points,points_patch,dataset1,dataset2,searching_window_w,searching_window_h,patch_w,patch_h,verbose=True,experiment=False,configs_ransac=None ):
    '''

        This code performs an image matching task with the given matcher, which takes two raw images and outputs corresponding features. The code first resizes the two raw images to (640, 480) and converts them to torch tensors, normalizing them by dividing each pixel by 255. The two images are then passed to the matcher to get feature matches and confidence scores.
        The code then computes two weighted average points based on the matches, one weighted by the confidence score and the other by a uniform weight. If the number of matches is greater than 0, the code returns the weighted average points and prints them as figures if verbose is set to True. The output figures are saved as "LoFTR-colab-demo.pdf".
I       f there are no matches, the code returns None, None, None, None.
'''


    img0 = torch.from_numpy(img0_raw)[None][None].cuda().to(torch.float) / 255.
    img1 = torch.from_numpy(img1_raw)[None][None].cuda().to(torch.float) / 255.

    batch = {'image0': img0, 'image1': img1}

    # Inference with LoFTR and get prediction
    with torch.no_grad():

            matcher(batch)
            mkpts0 = batch['mkpts0_f'].cpu().numpy()
            mkpts1= batch['mkpts1_f'].cpu().numpy()
            mconf = batch['mconf'].cpu().numpy()

            #mkpts0 = np.array([])

    results = []
    print()
    if mkpts0.shape[0]> 4:

        if configs_ransac is not None:

            for conf in configs_ransac:
                from skimage.measure import ransac
                if conf['residual_threshold']>= 0:
                    model, inliers = ransac((mkpts0, mkpts1),AffineTransform, min_samples=conf['min_samples'], residual_threshold=conf['residual_threshold'], max_trials=conf['max_trials'])
                    n_inliers = np.sum(inliers)
                    if n_inliers is None or n_inliers< 3:
                        conf.update({'rmse':-1,'inliers':0 if n_inliers is None else n_inliers})
                       # results.append({'rmse':-1,'inliers':})
                    else:
                        rmse = get_metrics(mkpts0[inliers], mkpts1[inliers],points,points_patch,dataset1,dataset2,searching_window_w,searching_window_h,patch_w,patch_h,rgb_img1,rgb_img2,mconf[inliers],verbose=verbose)
                else:
                     rmse= get_metrics(mkpts0, mkpts1,points,points_patch,dataset1,dataset2,searching_window_w,searching_window_h,patch_w,patch_h,rgb_img1,rgb_img2,mconf,verbose=verbose)
                     n_inliers = mkpts0.shape[0]
                conf.update({'rmse':rmse,'inliers':n_inliers})
                results.append(conf)
                if conf['residual_threshold'] == 1:

                    color = cm.jet(mconf, alpha=0.7)
                    text = [
                        'LoFTR',
                        'Matches: {}'.format(len(mkpts0)),
                    ]
                    if verbose:
                        #rgb_img1= np.swapaxes(rgb_img1,0,1)
                        #rgb_img2= np.swapaxes(rgb_img2,0,1)
                        abs_m0 = np.array([(x+points[1],y+points[0]) for x,y in mkpts0[inliers]])
                        abs_m1 = np.array([(x-((searching_window_h/2)-(patch_h/2))+points_patch[1] ,y-((searching_window_w/2)-(patch_w/2))+points_patch[0]) for x,y in mkpts1[inliers]])
                        fig = make_matching_figure(rgb_img1, rgb_img2, abs_m0, abs_m1, color[inliers], abs_m0, abs_m1, text)
                        make_matching_figure(rgb_img1, rgb_img2, abs_m0, abs_m1, color[inliers], abs_m0, abs_m1, text, path="LoFTR-colab-demo.pdf")

                        fig = make_matching_figure(img0_raw, img1_raw, mkpts0[inliers], mkpts1[inliers], color[inliers], mkpts0[inliers], mkpts1[inliers], text)
                        make_matching_figure(img0_raw, img1_raw, mkpts0[inliers], mkpts1[inliers], color[inliers], mkpts0[inliers], mkpts1[inliers], text, path="LoFTR-colab-demo.pdf")
        return results
    return None


In [5]:

from torch import nn
from torchvision.transforms import functional as F
import os
import numpy as np
import torch
from despekle.transform_main import TransSAR, TransSARV2, TransSARV3
import cv2

loaddirec = "model.pth"
save_path = "./"

device = torch.device("cuda")
if model_TransSAR is None:
    model_TransSAR = TransSARV2()
    model_TransSAR .to(device)
    model_TransSAR = nn.DataParallel(model_TransSAR ,device_ids=[0]).cuda()
    model_TransSAR.load_state_dict(torch.load(loaddirec))
    model_TransSAR.eval()


In [6]:
'''
 It takes two images as input, performs image denoising, and then uses a matcher to find the similarity between a patch from the first image and the second image.

The code uses a loop to run the processing for 50 times, for each iteration:

It calls the function get_sample to get the images, points, and datasets.
It denoises the raw images using 3 denoising methods (mean, bilateral, and lee_enhanced). If the denoising didn't produce a result, the code skips to the next iteration.
The code calls the function predict_and_print to get the result of the matcher and to find the similarity between the patch and the search window.
The code uses the result of the matcher to calculate the error in meters between the predicted position and the real position.
The code draws circles on the original full map to show the predicted and real positions.
The code displays the original full map if verbose is set to True.
The code accumulates the error for each iteration and prints the final result, which includes the number of successful predictions and the mean error in meters.
'''
def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output
def get_smoothed(file,savefile):
    im_file = file
    img = cv2.imread(im_file,0)

    #noisy_im = (np.float32(img)+1.0)/256.0

    #x = np.float32(noisy_im)
    #x = F.to_tensor(x)
    #x = x.unsqueeze(0)


    #pred_im = model_TransSAR(x)
    #tmp = pred_im.detach().cpu().numpy()

    #tmp = tmp.squeeze()
    #tmp = tmp*256 -1

    filepath_out = savefile

    cv2.imwrite(filepath_out,img)

def do_test(matcher_in,path0,path1,size_search=(640, 480),size_patch=(int(180*1.333333), int(180)),configs_ransac=None,verbose=True):


  rgb_img1, rgb_img2, points,points_patch_ref,points_patch,dataset1,dataset2 = get_sample(path0,path1,size_search,size_patch,verbose=verbose)
  if rgb_img1 is None:
      return None
  img0_pth = "./search_window.jpeg"
  img1_pth = "./patch.jpeg"
  get_smoothed("search_window.jpeg","search_window_s.jpeg")
  get_smoothed("patch.jpeg","patch_s.jpeg")
  image_pair = ["search_window_s.jpeg", "patch_s.jpeg"]
  img0_raw = cv2.imread(image_pair[0], cv2.IMREAD_GRAYSCALE)
  img1_raw = cv2.imread(image_pair[1], cv2.IMREAD_GRAYSCALE)

  img0_denoised =img0_raw
  img1_denoised = img1_raw

  results = predict_and_print(rgb_img1,rgb_img2,matcher_in,img0_denoised,img1_denoised,points,points_patch,dataset1,dataset2, size_search[0],size_search[1],size_patch[0],size_patch[1],verbose=verbose,configs_ransac=configs_ransac)

  return results

def get_list():
    path_of_the_directory= './data/paired_sentinel/'
    paths = []
    for filename in os.listdir(path_of_the_directory):
        f = os.path.join(path_of_the_directory,filename)
        if not os.path.isfile(f):
            lst = os.listdir(f)
            if len(lst)>1:
                paths.append((os.path.join(path_of_the_directory,filename,lst[0]),os.path.join(path_of_the_directory,filename,lst[1])))
    return paths

In [7]:


matcher_in=get_matcher("outdoor")
verbose=False
random.seed(9)
count_yes= 0
test_for_each_pair=1
configs_ransac=[{'min_samples':0,'residual_threshold':-1,'max_trials':0},
                {'min_samples':4,'residual_threshold':1,'max_trials':10000},
                      {'min_samples':4,'residual_threshold':3,'max_trials':10000},
                      {'min_samples':4,'residual_threshold':5,'max_trials':10000},
                      {'min_samples':4,'residual_threshold':10,'max_trials':10000},
                      {'min_samples':4,'residual_threshold':20,'max_trials':10000}]
paths = get_list()

metrics = [{'rmse':0,'inliers':0,'accepted_match':0} for conf in range(len(configs_ransac))]

for path0,path1 in paths:
    for x in range(test_for_each_pair):
              if random.randint(0, 1):
                results = do_test(matcher_in,path0,path1,size_search=(640, 480),size_patch=(int(180*1.333333)*2, int(180)*2),verbose=verbose,configs_ransac=configs_ransac)
              else:
                results = do_test(matcher_in,path1,path0,size_search=(640, 480),size_patch=(int(180*1.333333)*2, int(180)*2),verbose=verbose,configs_ransac=configs_ransac)

              if results is not None:

                for i in range(len(results)):
                    if metrics[i]['rmse']>=0:

                        if results[i]['inliers'] is not None:
                            metrics[i]['rmse'] += results[i]['rmse']
                            metrics[i]['inliers'] += results[i]['inliers']
                            metrics[i]['accepted_match'] +=1
                    print(results[i])

                count_yes+=1

data = []
for i in range(len(metrics)):
        metrics[i]['rmse'] /= metrics[i]['accepted_match']
        metrics[i]['inliers'] /= metrics[i]['accepted_match']

        configs_ransac[i]['rmse'] = metrics[i]['rmse']
        configs_ransac[i]['inliers'] = metrics[i]['inliers']
        configs_ransac[i]['Accepted_match'] = metrics[i]['accepted_match']
        configs_ransac[i]['total_match'] = test_for_each_pair*len(paths)
        data.append( configs_ransac[i])
            #print("Configuration:",configs_ransac[i-1]," Metrics:",metrics[i], " Accepted_match:", metrics[i]['accepted_match'] ,"/", test_for_each_pair*len(paths))




import pandas as pd



df = pd.DataFrame.from_dict(data)

print (df)

df.to_excel('players.xlsx')






{'min_samples': 0, 'residual_threshold': -1, 'max_trials': 0, 'rmse': 44.05519617958437, 'inliers': 31}
{'min_samples': 4, 'residual_threshold': 1, 'max_trials': 10000, 'rmse': 2.3776871592979085, 'inliers': 10}
{'min_samples': 4, 'residual_threshold': 3, 'max_trials': 10000, 'rmse': 3.061656666218341, 'inliers': 19}
{'min_samples': 4, 'residual_threshold': 5, 'max_trials': 10000, 'rmse': 3.39816205240859, 'inliers': 22}
{'min_samples': 4, 'residual_threshold': 10, 'max_trials': 10000, 'rmse': 4.5460565598035085, 'inliers': 26}
{'min_samples': 4, 'residual_threshold': 20, 'max_trials': 10000, 'rmse': 8.100829794563376, 'inliers': 27}


{'min_samples': 0, 'residual_threshold': -1, 'max_trials': 0, 'rmse': 111.44797515240582, 'inliers': 7}
{'min_samples': 4, 'residual_threshold': 1, 'max_trials': 10000, 'rmse': 111.44797515240582, 'inliers': 1}
{'min_samples': 4, 'residual_threshold': 3, 'max_trials': 10000, 'rmse': 111.44797515240582, 'inliers': 2}
{'min_samples': 4, 'residual_thresho

In [8]:
import pandas as pd



df = pd.DataFrame.from_dict(data)

print (df)

df.to_excel('players.xlsx')

   min_samples  residual_threshold  max_trials       rmse    inliers  \
0            0                  -1           0  81.708882  28.423729   
1            4                   1       10000  49.067086  10.773585   
2            4                   3       10000  53.301144  18.263158   
3            4                   5       10000  49.707076  21.327586   
4            4                  10       10000  55.684989  23.457627   
5            4                  20       10000  58.245181  24.355932   

   Accepted_match  total_match  
0              59           64  
1              53           64  
2              57           64  
3              58           64  
4              59           64  
5              59           64  
